In [1]:
import re

import pandas as pd

from sklearn import preprocessing

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import keras
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.manifold import TSNE

from bokeh.plotting import figure, ColumnDataSource, output_notebook, output_file, show, save 
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool
from bokeh.palettes import brewer
output_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


Loading BokehJS ...

In [2]:
# Input - type the file name
file = 'full_data_set_1.txt'

# loads the data
df1 = pd.read_table(file, delim_whitespace=True, names=['System', 'index', 'lattice_a',
                                                        'lattice_b','lattice_c','alfa',
                                                        'beta',
                                                        'gama',
                                                        'Ax (Cartesian)',
                                                        'Ay',
                                                        'Az',
                                                        'Bx',
                                                        'By',
                                                        'Bz',
                                                        'O1x',
                                                        'O1y',
                                                        'O1z',
                                                        'O2x',
                                                        'O2y',
                                                        'O2z',
                                                        'O3x',
                                                        'O3y',
                                                        'O3z',
                                                        'Energy',
                                                        'Force Ax (eV/A)',
                                                        'F Ay',
                                                        'F Az',
                                                        'F Bx',
                                                        'F By',
                                                        'F Bz',
                                                        'F O1x',
                                                        'F O1y',
                                                        'F O1z',
                                                        'F O2x',
                                                        'F O2y',
                                                        'F O2z',
                                                        'F O3x',
                                                        'F O3y',
                                                        'F O3z',
                                                        'sxx (eV/A^2)',
                                                        'syy',
                                                        'szz',
                                                        'syz',
                                                        'sxz',
                                                        'sxy'])

In [3]:
# displays the data
df1.loc[0]

System                 BaCaO3
index                       0
lattice_a             3.52716
lattice_b             4.05456
lattice_c             3.60249
alfa                  91.4972
beta                  91.3475
gama                  88.6236
Ax (Cartesian)              0
Ay                          0
Az                          0
Bx                  -0.239512
By                  -0.299065
Bz                -0.00400957
O1x                 0.0583789
O1y                 -0.161206
O1z                 -0.181566
O2x                  0.268339
O2y                 0.0605819
O2z                  0.169448
O3x                 0.0360643
O3y                  0.267463
O3z                -0.0372965
Energy               -23.3573
Force Ax (eV/A)      0.245684
F Ay                -0.325699
F Az                 -1.50125
F Bx                  8.39444
F By                  -2.4801
F Bz                  7.55875
F O1x                0.707234
F O1y                -2.50419
F O1z                 1.38106
F O2x     

In [4]:
# Checks the number of values for each sample
for i, value in enumerate(df1['System'].unique()):
    count = len(df1[df1.System == value])
    print(f'{i} {value} number of index = {count}')

0 BaCaO3 number of index = 1000
1 BaCoO3 number of index = 1000
2 BaCrO3 number of index = 1000
3 BaCuO3 number of index = 1000
4 BaFeO3 number of index = 1000
5 BaHfO3 number of index = 1000
6 BaMgO3 number of index = 1000
7 BaMnO3 number of index = 1000
8 BaMoO3 number of index = 1000
9 BaNbO3 number of index = 1000
10 BaNiO3 number of index = 1000
11 BaPbO3 number of index = 1000
12 BaScO3 number of index = 1000
13 BaSnO3 number of index = 1000
14 BaTiO3 number of index = 1000
15 BaVO3 number of index = 1000
16 BaZnO3 number of index = 1000
17 BaZrO3 number of index = 1000
18 BiCaO3 number of index = 1000
19 BiCoO3 number of index = 1000
20 BiCrO3 number of index = 1000
21 BiCuO3 number of index = 1000
22 BiFeO3 number of index = 1000
23 BiHfO3 number of index = 1000
24 BiMgO3 number of index = 1000
25 BiMnO3 number of index = 1000
26 BiMoO3 number of index = 1000
27 BiNbO3 number of index = 1000
28 BiNiO3 number of index = 1000
29 BiPbO3 number of index = 1000
30 BiScO3 number of i

In [5]:
# Converts the text labels into examples
encoder = LabelEncoder()
encoder.fit(df1['System'])
encoded_labels = encoder.transform(df1['System'])

# convert integers to dummy variables (i.e. one hot encoded)
labels = np_utils.to_categorical(encoded_labels)

# Collects the data for the samples
samples = df1.as_matrix(columns=df1.columns[2:39])

# Standard scaler to the raw data
samples_scaled=StandardScaler().fit_transform(samples)

In [6]:
#N.B. this assumes all of the a lattice values are unique

# Selects a random sampling of the data
# need to make sure there is no replacement
random_ind = np.random.choice(1000, 200, replace=False)

# Builds the Test and training set
test_ind = df1.loc[df1['index'].isin(random_ind)].index
train_ind = df1.loc[~df1['index'].isin(random_ind)].index

In [7]:
# First small model

# establishes the model
model = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model.add(Dense(512, input_dim=37))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 8
model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())
#
# Dense softmax layer for classification
model.add(Dense(126, activation='softmax'))

#opt = Adam()

# Compiles the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Full_graph', histogram_freq=0, write_graph=True, write_images=True)

#model.optimizer.lr.assign(0.0001)
hist = model.fit(samples_scaled[train_ind], labels[train_ind], epochs=5000, 
             batch_size=1000, validation_data = (samples_scaled[test_ind],labels[test_ind]),
                callbacks=[tbCallBack])

Train on 100800 samples, validate on 25200 samples
Epoch 1/5000
100800/100800 [==============================] - 3s - loss: 4.6713 - acc: 0.0184 - val_loss: 4.2995 - val_acc: 0.0250
Epoch 2/5000
100800/100800 [==============================] - 2s - loss: 4.0568 - acc: 0.0426 - val_loss: 3.9941 - val_acc: 0.0292
Epoch 3/5000
100800/100800 [==============================] - 2s - loss: 3.7076 - acc: 0.0588 - val_loss: 3.9227 - val_acc: 0.0338
Epoch 4/5000
100800/100800 [==============================] - 2s - loss: 3.5173 - acc: 0.0719 - val_loss: 3.5293 - val_acc: 0.0555
Epoch 5/5000
100800/100800 [==============================] - 2s - loss: 3.3753 - acc: 0.0909 - val_loss: 3.4879 - val_acc: 0.0638
Epoch 6/5000
100800/100800 [==============================] - 2s - loss: 3.2399 - acc: 0.1097 - val_loss: 3.2390 - val_acc: 0.1024
Epoch 7/5000
100800/100800 [==============================] - 2s - loss: 3.1197 - acc: 0.1294 - val_loss: 3.0408 - val_acc: 0.1396
Epoch 8/5000
100800/100800 [====

100800/100800 [==============================] - 2s - loss: 1.6630 - acc: 0.4401 - val_loss: 2.3281 - val_acc: 0.3600
Epoch 124/5000
100800/100800 [==============================] - 2s - loss: 1.6622 - acc: 0.4401 - val_loss: 2.3449 - val_acc: 0.3604
Epoch 125/5000
100800/100800 [==============================] - 2s - loss: 1.6599 - acc: 0.4423 - val_loss: 2.2467 - val_acc: 0.3842
Epoch 126/5000
100800/100800 [==============================] - 2s - loss: 1.6587 - acc: 0.4421 - val_loss: 2.3089 - val_acc: 0.3718
Epoch 127/5000
100800/100800 [==============================] - 2s - loss: 1.6567 - acc: 0.4427 - val_loss: 2.2742 - val_acc: 0.3771
Epoch 128/5000
100800/100800 [==============================] - 2s - loss: 1.6618 - acc: 0.4412 - val_loss: 2.2347 - val_acc: 0.3813
Epoch 129/5000
100800/100800 [==============================] - 2s - loss: 1.6539 - acc: 0.4433 - val_loss: 2.2252 - val_acc: 0.3861
Epoch 130/5000
100800/100800 [==============================] - 2s - loss: 1.6469 - 

100800/100800 [==============================] - 2s - loss: 1.5212 - acc: 0.4826 - val_loss: 2.3169 - val_acc: 0.3805
Epoch 246/5000
100800/100800 [==============================] - 2s - loss: 1.5228 - acc: 0.4790 - val_loss: 2.3834 - val_acc: 0.3664
Epoch 247/5000
100800/100800 [==============================] - 2s - loss: 1.5009 - acc: 0.4880 - val_loss: 2.3421 - val_acc: 0.3795
Epoch 248/5000
100800/100800 [==============================] - 2s - loss: 1.5387 - acc: 0.4756 - val_loss: 2.3250 - val_acc: 0.3825
Epoch 249/5000
100800/100800 [==============================] - 2s - loss: 1.5208 - acc: 0.4811 - val_loss: 2.3697 - val_acc: 0.3740
Epoch 250/5000
100800/100800 [==============================] - 2s - loss: 1.5142 - acc: 0.4823 - val_loss: 2.4256 - val_acc: 0.3623
Epoch 251/5000
100800/100800 [==============================] - 2s - loss: 1.5139 - acc: 0.4811 - val_loss: 2.4770 - val_acc: 0.3486
Epoch 252/5000
100800/100800 [==============================] - 2s - loss: 1.5144 - 

100800/100800 [==============================] - 2s - loss: 1.4434 - acc: 0.5008 - val_loss: 2.4005 - val_acc: 0.3660
Epoch 367/5000
100800/100800 [==============================] - 2s - loss: 1.4437 - acc: 0.5026 - val_loss: 2.4914 - val_acc: 0.3541
Epoch 368/5000
100800/100800 [==============================] - 2s - loss: 1.4495 - acc: 0.5018 - val_loss: 2.4059 - val_acc: 0.3729
Epoch 369/5000
100800/100800 [==============================] - 2s - loss: 1.4387 - acc: 0.5050 - val_loss: 2.3561 - val_acc: 0.3789
Epoch 370/5000
100800/100800 [==============================] - 2s - loss: 1.4513 - acc: 0.5012 - val_loss: 2.4536 - val_acc: 0.3659
Epoch 371/5000
100800/100800 [==============================] - 2s - loss: 1.4499 - acc: 0.5013 - val_loss: 2.5012 - val_acc: 0.3559
Epoch 372/5000
100800/100800 [==============================] - 2s - loss: 1.4337 - acc: 0.5072 - val_loss: 2.4411 - val_acc: 0.3667
Epoch 373/5000
100800/100800 [==============================] - 2s - loss: 1.4368 - 

100800/100800 [==============================] - 2s - loss: 1.3905 - acc: 0.5182 - val_loss: 2.4482 - val_acc: 0.3744
Epoch 488/5000
100800/100800 [==============================] - 2s - loss: 1.3919 - acc: 0.5199 - val_loss: 2.5649 - val_acc: 0.3527
Epoch 489/5000
100800/100800 [==============================] - 2s - loss: 1.3986 - acc: 0.5173 - val_loss: 2.4988 - val_acc: 0.3668
Epoch 490/5000
100800/100800 [==============================] - 2s - loss: 1.4045 - acc: 0.5171 - val_loss: 2.5705 - val_acc: 0.3562
Epoch 491/5000
100800/100800 [==============================] - 2s - loss: 1.3912 - acc: 0.5181 - val_loss: 2.6280 - val_acc: 0.34540.
Epoch 492/5000
100800/100800 [==============================] - 2s - loss: 1.3896 - acc: 0.5202 - val_loss: 2.4287 - val_acc: 0.3807
Epoch 493/5000
100800/100800 [==============================] - 2s - loss: 1.3858 - acc: 0.5200 - val_loss: 2.4782 - val_acc: 0.3678
Epoch 494/5000
100800/100800 [==============================] - 2s - loss: 1.3790 

100800/100800 [==============================] - 2s - loss: 1.3575 - acc: 0.5319 - val_loss: 2.5569 - val_acc: 0.3587
Epoch 610/5000
100800/100800 [==============================] - 2s - loss: 1.3606 - acc: 0.5270 - val_loss: 2.5036 - val_acc: 0.3632
Epoch 611/5000
100800/100800 [==============================] - 2s - loss: 1.3566 - acc: 0.5324 - val_loss: 2.7015 - val_acc: 0.34540 - ETA: 0s - loss: 1.3550 - acc:
Epoch 612/5000
100800/100800 [==============================] - 2s - loss: 1.3607 - acc: 0.5284 - val_loss: 2.5845 - val_acc: 0.3579
Epoch 613/5000
100800/100800 [==============================] - 2s - loss: 1.3513 - acc: 0.5310 - val_loss: 2.4755 - val_acc: 0.3744
Epoch 614/5000
100800/100800 [==============================] - 2s - loss: 1.3528 - acc: 0.5310 - val_loss: 2.4897 - val_acc: 0.3708
Epoch 615/5000
100800/100800 [==============================] - 2s - loss: 1.3621 - acc: 0.5289 - val_loss: 2.6205 - val_acc: 0.3540
Epoch 616/5000
100800/100800 [=====================

100800/100800 [==============================] - 2s - loss: 1.3485 - acc: 0.5324 - val_loss: 2.4482 - val_acc: 0.3739
Epoch 671/5000
100800/100800 [==============================] - 2s - loss: 1.3421 - acc: 0.5335 - val_loss: 2.7562 - val_acc: 0.3430
Epoch 672/5000
100800/100800 [==============================] - 2s - loss: 1.3540 - acc: 0.5313 - val_loss: 2.5277 - val_acc: 0.3652
Epoch 673/5000
100800/100800 [==============================] - 2s - loss: 1.3398 - acc: 0.5348 - val_loss: 2.5712 - val_acc: 0.3663
Epoch 674/5000
100800/100800 [==============================] - 2s - loss: 1.3578 - acc: 0.5307 - val_loss: 2.7261 - val_acc: 0.3406
Epoch 675/5000
100800/100800 [==============================] - 2s - loss: 1.3483 - acc: 0.5328 - val_loss: 2.4434 - val_acc: 0.3827
Epoch 676/5000
100800/100800 [==============================] - 2s - loss: 1.3422 - acc: 0.5350 - val_loss: 2.7300 - val_acc: 0.3374
Epoch 677/5000
100800/100800 [==============================] - 2s - loss: 1.3387 - 

100800/100800 [==============================] - 2s - loss: 1.3192 - acc: 0.5423 - val_loss: 2.5803 - val_acc: 0.3660
Epoch 793/5000
100800/100800 [==============================] - 2s - loss: 1.3092 - acc: 0.5450 - val_loss: 3.0027 - val_acc: 0.3154
Epoch 794/5000
100800/100800 [==============================] - 2s - loss: 1.3211 - acc: 0.5440 - val_loss: 2.5707 - val_acc: 0.3583
Epoch 795/5000
100800/100800 [==============================] - 2s - loss: 1.3212 - acc: 0.5416 - val_loss: 2.5616 - val_acc: 0.3649
Epoch 796/5000
100800/100800 [==============================] - 2s - loss: 1.3214 - acc: 0.5419 - val_loss: 2.5500 - val_acc: 0.3658
Epoch 797/5000
100800/100800 [==============================] - 2s - loss: 1.3304 - acc: 0.5380 - val_loss: 2.6575 - val_acc: 0.3414
Epoch 798/5000
100800/100800 [==============================] - 2s - loss: 1.3115 - acc: 0.5447 - val_loss: 2.6692 - val_acc: 0.3498
Epoch 799/5000
100800/100800 [==============================] - 2s - loss: 1.3105 - 

100800/100800 [==============================] - 2s - loss: 1.2960 - acc: 0.5484 - val_loss: 2.7421 - val_acc: 0.3398
Epoch 915/5000
100800/100800 [==============================] - 2s - loss: 1.2864 - acc: 0.5520 - val_loss: 2.7570 - val_acc: 0.3483
Epoch 916/5000
100800/100800 [==============================] - 2s - loss: 1.3081 - acc: 0.5458 - val_loss: 2.7510 - val_acc: 0.3431
Epoch 917/5000
100800/100800 [==============================] - 2s - loss: 1.2975 - acc: 0.5491 - val_loss: 2.8874 - val_acc: 0.3271
Epoch 918/5000
100800/100800 [==============================] - 2s - loss: 1.2988 - acc: 0.5488 - val_loss: 2.6495 - val_acc: 0.3565
Epoch 919/5000
100800/100800 [==============================] - 2s - loss: 1.2873 - acc: 0.5526 - val_loss: 2.7579 - val_acc: 0.3443
Epoch 920/5000
100800/100800 [==============================] - 2s - loss: 1.2899 - acc: 0.5514 - val_loss: 2.6804 - val_acc: 0.3546
Epoch 921/5000
100800/100800 [==============================] - 2s - loss: 1.2959 - 

KeyboardInterrupt: 

In [12]:
model = model.save('./full_test/_network_1_1000ep_56_35')

### model without dropout could not overfit the data well.
Try a network with bigger size ... then scale back

In [19]:
# First small model

# establishes the model
model = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model.add(Dense(1024, input_dim=37))
#model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 32
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 8
model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())
#
# Dense softmax layer for classification
model.add(Dense(126, activation='softmax'))

#opt = Adam()

# Compiles the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Full_graph', histogram_freq=0, write_graph=True, write_images=True)

#model.optimizer.lr.assign(0.0001)
hist = model.fit(samples_scaled[train_ind], labels[train_ind], epochs=1000, 
             batch_size=1000, validation_data = (samples_scaled[test_ind],labels[test_ind]),
                callbacks=[tbCallBack])

Train on 100800 samples, validate on 25200 samples
Epoch 1/1000
100800/100800 [==============================] - 2s - loss: 4.5144 - acc: 0.0293 - val_loss: 4.4181 - val_acc: 0.0156
Epoch 2/1000
100800/100800 [==============================] - 2s - loss: 3.8042 - acc: 0.0731 - val_loss: 4.0777 - val_acc: 0.0238
Epoch 3/1000
100800/100800 [==============================] - 2s - loss: 3.2783 - acc: 0.1565 - val_loss: 3.3435 - val_acc: 0.1238
Epoch 4/1000
100800/100800 [==============================] - 2s - loss: 2.7253 - acc: 0.2641 - val_loss: 3.3406 - val_acc: 0.1231
Epoch 5/1000
100800/100800 [==============================] - 2s - loss: 2.2704 - acc: 0.3683 - val_loss: 3.2233 - val_acc: 0.1585
Epoch 6/1000
100800/100800 [==============================] - 2s - loss: 1.9286 - acc: 0.4521 - val_loss: 3.2581 - val_acc: 0.1748
Epoch 7/1000
100800/100800 [==============================] - 2s - loss: 1.6843 - acc: 0.5121 - val_loss: 3.2305 - val_acc: 0.1894
Epoch 8/1000
100800/100800 [====

KeyboardInterrupt: 

In [21]:
model = model.save('./full_test/network_2_1000ep_93_40_no_drop')

In [29]:
# First small model
i = 10

# establishes the model
model = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model.add(Dense(1024, input_dim=37))
#model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 32
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())

for i in range(i):
    # Dense layer size 32
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 8
model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())
#
# Dense softmax layer for classification
model.add(Dense(126, activation='softmax'))

#opt = Adam()

# Compiles the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Full_graph', histogram_freq=0, write_graph=True, write_images=True)

#model.optimizer.lr.assign(0.0001)
hist = model.fit(samples_scaled[train_ind], labels[train_ind], epochs=1000, 
             batch_size=1000, validation_data = (samples_scaled[test_ind],labels[test_ind]),
                callbacks=[tbCallBack])

Train on 100800 samples, validate on 25200 samples
Epoch 1/1000
100800/100800 [==============================] - 6s - loss: 4.8951 - acc: 0.0086 - val_loss: 4.8544 - val_acc: 0.0080
Epoch 2/1000
100800/100800 [==============================] - 5s - loss: 4.8060 - acc: 0.0124 - val_loss: 5.0762 - val_acc: 0.0092
Epoch 3/1000
100800/100800 [==============================] - 5s - loss: 4.4047 - acc: 0.0241 - val_loss: 5.0191 - val_acc: 0.0194
Epoch 4/1000
100800/100800 [==============================] - 5s - loss: 3.9112 - acc: 0.0457 - val_loss: 4.2907 - val_acc: 0.0362
Epoch 5/1000
100800/100800 [==============================] - 5s - loss: 3.5724 - acc: 0.0619 - val_loss: 4.6110 - val_acc: 0.0246
Epoch 6/1000
100800/100800 [==============================] - 5s - loss: 3.3650 - acc: 0.0742 - val_loss: 4.9361 - val_acc: 0.0246
Epoch 7/1000
100800/100800 [==============================] - 5s - loss: 3.2390 - acc: 0.0838 - val_loss: 4.9447 - val_acc: 0.0298
Epoch 8/1000
100800/100800 [====

100800/100800 [==============================] - 5s - loss: 0.7527 - acc: 0.7555 - val_loss: 4.1301 - val_acc: 0.3460
Epoch 124/1000
100800/100800 [==============================] - 5s - loss: 0.7262 - acc: 0.7631 - val_loss: 3.9838 - val_acc: 0.3638
Epoch 125/1000
100800/100800 [==============================] - 5s - loss: 0.7294 - acc: 0.7626 - val_loss: 4.0248 - val_acc: 0.3562
Epoch 126/1000
100800/100800 [==============================] - 5s - loss: 0.7269 - acc: 0.7646 - val_loss: 4.0999 - val_acc: 0.3584
Epoch 127/1000
100800/100800 [==============================] - 5s - loss: 0.7152 - acc: 0.7666 - val_loss: 4.1656 - val_acc: 0.3493
Epoch 128/1000
100800/100800 [==============================] - 5s - loss: 0.7185 - acc: 0.7663 - val_loss: 4.1799 - val_acc: 0.3590
Epoch 129/1000
100800/100800 [==============================] - 5s - loss: 0.7265 - acc: 0.7645 - val_loss: 4.1089 - val_acc: 0.3527
Epoch 130/1000
100800/100800 [==============================] - 5s - loss: 0.7244 - 

100800/100800 [==============================] - 5s - loss: 0.4789 - acc: 0.8490 - val_loss: 4.5571 - val_acc: 0.3629
Epoch 246/1000
100800/100800 [==============================] - 5s - loss: 0.4580 - acc: 0.8540 - val_loss: 4.6286 - val_acc: 0.3576
Epoch 247/1000
100800/100800 [==============================] - 5s - loss: 0.4497 - acc: 0.8575 - val_loss: 4.5311 - val_acc: 0.3634
Epoch 248/1000
100800/100800 [==============================] - 5s - loss: 0.4665 - acc: 0.8520 - val_loss: 4.6257 - val_acc: 0.3629
Epoch 249/1000
100800/100800 [==============================] - 5s - loss: 0.4491 - acc: 0.8577 - val_loss: 4.5013 - val_acc: 0.3711
Epoch 250/1000
100800/100800 [==============================] - 5s - loss: 0.4513 - acc: 0.8581 - val_loss: 4.6835 - val_acc: 0.3536
Epoch 251/1000
100800/100800 [==============================] - 5s - loss: 0.4673 - acc: 0.8531 - val_loss: 4.6842 - val_acc: 0.3579
Epoch 252/1000
100800/100800 [==============================] - 5s - loss: 0.4681 - 

100800/100800 [==============================] - 5s - loss: 0.3495 - acc: 0.8908 - val_loss: 4.8296 - val_acc: 0.3663
Epoch 368/1000
100800/100800 [==============================] - 5s - loss: 0.3635 - acc: 0.8877 - val_loss: 4.8644 - val_acc: 0.3588
Epoch 369/1000
100800/100800 [==============================] - 5s - loss: 0.3780 - acc: 0.8828 - val_loss: 4.7941 - val_acc: 0.3660
Epoch 370/1000
100800/100800 [==============================] - 5s - loss: 0.3539 - acc: 0.8889 - val_loss: 4.8576 - val_acc: 0.3638
Epoch 371/1000
100800/100800 [==============================] - 5s - loss: 0.3468 - acc: 0.8908 - val_loss: 4.7815 - val_acc: 0.3651
Epoch 372/1000
100800/100800 [==============================] - 5s - loss: 0.3518 - acc: 0.8906 - val_loss: 4.7752 - val_acc: 0.3643
Epoch 373/1000
100800/100800 [==============================] - 5s - loss: 0.3669 - acc: 0.8851 - val_loss: 4.9135 - val_acc: 0.3538
Epoch 374/1000
100800/100800 [==============================] - 5s - loss: 0.3510 - 

100800/100800 [==============================] - 5s - loss: 0.2998 - acc: 0.9080 - val_loss: 4.9486 - val_acc: 0.3632
Epoch 490/1000
100800/100800 [==============================] - 5s - loss: 0.2926 - acc: 0.9098 - val_loss: 5.0790 - val_acc: 0.3530
Epoch 491/1000
100800/100800 [==============================] - 5s - loss: 0.2843 - acc: 0.9121 - val_loss: 5.0087 - val_acc: 0.3497
Epoch 492/1000
100800/100800 [==============================] - 5s - loss: 0.3018 - acc: 0.9067 - val_loss: 5.1077 - val_acc: 0.35690.9
Epoch 493/1000
100800/100800 [==============================] - 5s - loss: 0.2999 - acc: 0.9070 - val_loss: 5.1583 - val_acc: 0.3518
Epoch 494/1000
100800/100800 [==============================] - 5s - loss: 0.2911 - acc: 0.9101 - val_loss: 5.1454 - val_acc: 0.3575
Epoch 495/1000
100800/100800 [==============================] - 5s - loss: 0.2842 - acc: 0.9129 - val_loss: 5.0482 - val_acc: 0.3573
Epoch 496/1000
100800/100800 [==============================] - 5s - loss: 0.2888

KeyboardInterrupt: 

In [31]:
model = model.save('./full_test/network_3_600ep_93_35_no_drop')

In [56]:
# First small model

# establishes the model
model = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model.add(Dense(1024, input_dim=37))
model.add(Dropout(0.5))
model.add(BatchNormalization())


# Dense layer size 32
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Dense layer size 8
model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())
#
# Dense softmax layer for classification
model.add(Dense(126, activation='softmax'))

#opt = Adam()

# Compiles the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Full_graph', histogram_freq=0, write_graph=True, write_images=True)

#model.optimizer.lr.assign(0.0001)
hist = model.fit(samples_scaled[train_ind], labels[train_ind], epochs=1000, 
             batch_size=20000, validation_data = (samples_scaled[test_ind],labels[test_ind]),
                callbacks=[tbCallBack])

Train on 100800 samples, validate on 25200 samples
Epoch 1/1000
100800/100800 [==============================] - 0s - loss: 2.6165 - acc: 0.1776 - val_loss: 2.6543 - val_acc: 0.2116
Epoch 2/1000
100800/100800 [==============================] - 0s - loss: 2.6195 - acc: 0.1785 - val_loss: 2.6183 - val_acc: 0.2145
Epoch 3/1000
100800/100800 [==============================] - 0s - loss: 2.6274 - acc: 0.1795 - val_loss: 2.6313 - val_acc: 0.2121
Epoch 4/1000
100800/100800 [==============================] - 0s - loss: 2.6239 - acc: 0.1778 - val_loss: 2.6191 - val_acc: 0.2145
Epoch 5/1000
100800/100800 [==============================] - 0s - loss: 2.6268 - acc: 0.1772 - val_loss: 2.6037 - val_acc: 0.2152
Epoch 6/1000
100800/100800 [==============================] - 0s - loss: 2.6235 - acc: 0.1782 - val_loss: 2.6513 - val_acc: 0.2091
Epoch 7/1000
100800/100800 [==============================] - 0s - loss: 2.6195 - acc: 0.1780 - val_loss: 2.6147 - val_acc: 0.2140
Epoch 8/1000
100800/100800 [====

100800/100800 [==============================] - 0s - loss: 2.6012 - acc: 0.1811 - val_loss: 2.6041 - val_acc: 0.2180
Epoch 124/1000
100800/100800 [==============================] - 0s - loss: 2.5958 - acc: 0.1821 - val_loss: 2.6025 - val_acc: 0.2187
Epoch 125/1000
100800/100800 [==============================] - 0s - loss: 2.5977 - acc: 0.1831 - val_loss: 2.6430 - val_acc: 0.2057
Epoch 126/1000
100800/100800 [==============================] - 0s - loss: 2.5960 - acc: 0.1803 - val_loss: 2.6486 - val_acc: 0.2061
Epoch 127/1000
100800/100800 [==============================] - 0s - loss: 2.5916 - acc: 0.1837 - val_loss: 2.6267 - val_acc: 0.2152
Epoch 128/1000
100800/100800 [==============================] - 0s - loss: 2.5949 - acc: 0.1813 - val_loss: 2.5997 - val_acc: 0.2143
Epoch 129/1000
100800/100800 [==============================] - 0s - loss: 2.5947 - acc: 0.1832 - val_loss: 2.6161 - val_acc: 0.2123
Epoch 130/1000
100800/100800 [==============================] - 0s - loss: 2.5892 - 

100800/100800 [==============================] - 0s - loss: 2.5788 - acc: 0.1849 - val_loss: 2.5861 - val_acc: 0.2269
Epoch 246/1000
100800/100800 [==============================] - 0s - loss: 2.5844 - acc: 0.1854 - val_loss: 2.5873 - val_acc: 0.2256
Epoch 247/1000
100800/100800 [==============================] - 0s - loss: 2.5792 - acc: 0.1870 - val_loss: 2.5833 - val_acc: 0.22680.
Epoch 248/1000
100800/100800 [==============================] - 0s - loss: 2.5727 - acc: 0.1891 - val_loss: 2.6411 - val_acc: 0.2188
Epoch 249/1000
100800/100800 [==============================] - 0s - loss: 2.5817 - acc: 0.1860 - val_loss: 2.6399 - val_acc: 0.2152
Epoch 250/1000
100800/100800 [==============================] - 0s - loss: 2.5746 - acc: 0.1883 - val_loss: 2.6153 - val_acc: 0.2175
Epoch 251/1000
100800/100800 [==============================] - 0s - loss: 2.5653 - acc: 0.1894 - val_loss: 2.5993 - val_acc: 0.2197
Epoch 252/1000
100800/100800 [==============================] - 0s - loss: 2.5715 

100800/100800 [==============================] - 0s - loss: 2.5544 - acc: 0.1947 - val_loss: 2.5742 - val_acc: 0.2350
Epoch 368/1000
100800/100800 [==============================] - 0s - loss: 2.5435 - acc: 0.1965 - val_loss: 2.5484 - val_acc: 0.2357
Epoch 369/1000
100800/100800 [==============================] - 0s - loss: 2.5392 - acc: 0.1980 - val_loss: 2.5921 - val_acc: 0.2249
Epoch 370/1000
100800/100800 [==============================] - 0s - loss: 2.5372 - acc: 0.1988 - val_loss: 2.5637 - val_acc: 0.2325
Epoch 371/1000
100800/100800 [==============================] - 0s - loss: 2.5383 - acc: 0.1990 - val_loss: 2.5671 - val_acc: 0.2338
Epoch 372/1000
100800/100800 [==============================] - 0s - loss: 2.5367 - acc: 0.1980 - val_loss: 2.5596 - val_acc: 0.2385
Epoch 373/1000
100800/100800 [==============================] - 0s - loss: 2.5328 - acc: 0.1991 - val_loss: 2.5890 - val_acc: 0.2304
Epoch 374/1000
100800/100800 [==============================] - 0s - loss: 2.5307 - 

100800/100800 [==============================] - 0s - loss: 2.5008 - acc: 0.2135 - val_loss: 2.5696 - val_acc: 0.2411
Epoch 490/1000
100800/100800 [==============================] - 0s - loss: 2.5010 - acc: 0.2144 - val_loss: 2.5610 - val_acc: 0.2469
Epoch 491/1000
100800/100800 [==============================] - 0s - loss: 2.5006 - acc: 0.2133 - val_loss: 2.5346 - val_acc: 0.2482
Epoch 492/1000
100800/100800 [==============================] - 0s - loss: 2.5048 - acc: 0.2119 - val_loss: 2.5241 - val_acc: 0.2490
Epoch 493/1000
100800/100800 [==============================] - 0s - loss: 2.4994 - acc: 0.2148 - val_loss: 2.5146 - val_acc: 0.2533
Epoch 494/1000
100800/100800 [==============================] - 0s - loss: 2.5006 - acc: 0.2140 - val_loss: 2.4976 - val_acc: 0.2566
Epoch 495/1000
100800/100800 [==============================] - 0s - loss: 2.4959 - acc: 0.2168 - val_loss: 2.5193 - val_acc: 0.2540
Epoch 496/1000
100800/100800 [==============================] - 0s - loss: 2.5002 - 

100800/100800 [==============================] - 0s - loss: 2.4578 - acc: 0.2283 - val_loss: 2.5122 - val_acc: 0.2620
Epoch 612/1000
100800/100800 [==============================] - 0s - loss: 2.4501 - acc: 0.2296 - val_loss: 2.5312 - val_acc: 0.2595
Epoch 613/1000
100800/100800 [==============================] - 0s - loss: 2.4693 - acc: 0.2268 - val_loss: 2.5404 - val_acc: 0.2567
Epoch 614/1000
100800/100800 [==============================] - 0s - loss: 2.4617 - acc: 0.2260 - val_loss: 2.4602 - val_acc: 0.2707
Epoch 615/1000
100800/100800 [==============================] - 0s - loss: 2.4553 - acc: 0.2318 - val_loss: 2.5113 - val_acc: 0.2626
Epoch 616/1000
100800/100800 [==============================] - 0s - loss: 2.4610 - acc: 0.2293 - val_loss: 2.4811 - val_acc: 0.2662
Epoch 617/1000
100800/100800 [==============================] - 0s - loss: 2.4656 - acc: 0.2274 - val_loss: 2.4261 - val_acc: 0.2795
Epoch 618/1000
100800/100800 [==============================] - 0s - loss: 2.4541 - 

100800/100800 [==============================] - 0s - loss: 2.4123 - acc: 0.2430 - val_loss: 2.3995 - val_acc: 0.2858
Epoch 734/1000
100800/100800 [==============================] - 0s - loss: 2.4183 - acc: 0.2392 - val_loss: 2.4492 - val_acc: 0.2769
Epoch 735/1000
100800/100800 [==============================] - 0s - loss: 2.4258 - acc: 0.2405 - val_loss: 2.4216 - val_acc: 0.2807
Epoch 736/1000
100800/100800 [==============================] - 0s - loss: 2.4158 - acc: 0.2407 - val_loss: 2.3832 - val_acc: 0.2877
Epoch 737/1000
100800/100800 [==============================] - 0s - loss: 2.4175 - acc: 0.2426 - val_loss: 2.3870 - val_acc: 0.2860
Epoch 738/1000
100800/100800 [==============================] - 0s - loss: 2.4114 - acc: 0.2423 - val_loss: 2.4252 - val_acc: 0.2820
Epoch 739/1000
100800/100800 [==============================] - 0s - loss: 2.4190 - acc: 0.2399 - val_loss: 2.4227 - val_acc: 0.2862
Epoch 740/1000
100800/100800 [==============================] - 0s - loss: 2.4075 - 

100800/100800 [==============================] - 0s - loss: 2.3830 - acc: 0.2505 - val_loss: 2.4052 - val_acc: 0.2902
Epoch 856/1000
100800/100800 [==============================] - 0s - loss: 2.3860 - acc: 0.2517 - val_loss: 2.4361 - val_acc: 0.2860
Epoch 857/1000
100800/100800 [==============================] - 0s - loss: 2.3811 - acc: 0.2527 - val_loss: 2.4200 - val_acc: 0.2883
Epoch 858/1000
100800/100800 [==============================] - 0s - loss: 2.3823 - acc: 0.2530 - val_loss: 2.4404 - val_acc: 0.2787
Epoch 859/1000
100800/100800 [==============================] - 0s - loss: 2.3801 - acc: 0.2529 - val_loss: 2.4076 - val_acc: 0.2859
Epoch 860/1000
100800/100800 [==============================] - 0s - loss: 2.3755 - acc: 0.2522 - val_loss: 2.3954 - val_acc: 0.2881
Epoch 861/1000
100800/100800 [==============================] - 0s - loss: 2.3749 - acc: 0.2539 - val_loss: 2.3906 - val_acc: 0.2922
Epoch 862/1000
100800/100800 [==============================] - 0s - loss: 2.3863 - 

100800/100800 [==============================] - 0s - loss: 2.3600 - acc: 0.2570 - val_loss: 2.3756 - val_acc: 0.2992
Epoch 978/1000
100800/100800 [==============================] - 0s - loss: 2.3595 - acc: 0.2568 - val_loss: 2.3626 - val_acc: 0.3082
Epoch 979/1000
100800/100800 [==============================] - 0s - loss: 2.3544 - acc: 0.2595 - val_loss: 2.3807 - val_acc: 0.3028
Epoch 980/1000
100800/100800 [==============================] - 0s - loss: 2.3562 - acc: 0.2594 - val_loss: 2.3922 - val_acc: 0.2992
Epoch 981/1000
100800/100800 [==============================] - 0s - loss: 2.3550 - acc: 0.2614 - val_loss: 2.3597 - val_acc: 0.2991
Epoch 982/1000
100800/100800 [==============================] - 0s - loss: 2.3630 - acc: 0.2587 - val_loss: 2.3584 - val_acc: 0.3029
Epoch 983/1000
100800/100800 [==============================] - 0s - loss: 2.3575 - acc: 0.2602 - val_loss: 2.3449 - val_acc: 0.3052
Epoch 984/1000
100800/100800 [==============================] - 0s - loss: 2.3500 - 

In [61]:
model.save('./full_test/network_6_2000ep_26_29')

In [44]:
model = keras.models.load_model('./full_test/network_5_1000ep_1000_32_35')

In [62]:
model.evaluate(samples_scaled,labels)

125728/126000 [============================>.] - ETA: 0s

[1.9780224759238108, 0.38607142857142857]

In [93]:
# First small model

dropout = 0.5

# establishes the model
model = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model.add(Dense(np.ceil(1024*(1+3*dropout)).astype(int), input_dim=37))
model.add(Dropout(dropout))
model.add(BatchNormalization())


# Dense layer size 32
model.add(Dense(np.ceil(512*(1+3*dropout)).astype(int), activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(np.ceil(512*(1+3*dropout)).astype(int), activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())

# Dense layer size 32
model.add(Dense(np.ceil(256*(1+3*dropout)).astype(int), activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(np.ceil(128*(1+3*dropout)).astype(int), activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())

# Dense layer size 8
model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 16
model.add(Dense(np.ceil(256*(1+3*dropout)).astype(int), activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())
#
# Dense softmax layer for classification
model.add(Dense(126, activation='softmax'))

#opt = Adam()

# Compiles the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Full_graph', histogram_freq=0, write_graph=True, write_images=True)
for i in range(50):
    #model.optimizer.lr.assign(0.0001)
    hist = model.fit(samples_scaled[train_ind], labels[train_ind], epochs=1000, 
                 batch_size=60000, validation_data = (samples_scaled[test_ind],labels[test_ind]),
                    callbacks=[tbCallBack])
    epocher = i*1000+51001
    model.save(f'./full_test/network_11_{epocher}ep_59_4')

Train on 100800 samples, validate on 25200 samples
Epoch 1/1000
100800/100800 [==============================] - 1s - loss: 0.6404 - acc: 0.7740 - val_loss: 2.9908 - val_acc: 0.4518
Epoch 2/1000
100800/100800 [==============================] - 1s - loss: 0.6420 - acc: 0.7729 - val_loss: 3.0073 - val_acc: 0.4507
Epoch 3/1000
100800/100800 [==============================] - 1s - loss: 0.6375 - acc: 0.7766 - val_loss: 2.9984 - val_acc: 0.4514
Epoch 4/1000
100800/100800 [==============================] - 1s - loss: 0.6436 - acc: 0.7729 - val_loss: 2.9739 - val_acc: 0.4535
Epoch 5/1000
100800/100800 [==============================] - 1s - loss: 0.6384 - acc: 0.7742 - val_loss: 2.9633 - val_acc: 0.4545
Epoch 6/1000
100800/100800 [==============================] - 1s - loss: 0.6375 - acc: 0.7760 - val_loss: 2.9770 - val_acc: 0.4519
Epoch 7/1000
100800/100800 [==============================] - 1s - loss: 0.6409 - acc: 0.7750 - val_loss: 3.0017 - val_acc: 0.4511
Epoch 8/1000
100800/100800 [====

100800/100800 [==============================] - 1s - loss: 0.6481 - acc: 0.7711 - val_loss: 2.9792 - val_acc: 0.4565
Epoch 124/1000
100800/100800 [==============================] - 1s - loss: 0.6504 - acc: 0.7704 - val_loss: 2.9831 - val_acc: 0.4576
Epoch 125/1000
100800/100800 [==============================] - 1s - loss: 0.6488 - acc: 0.7710 - val_loss: 3.0055 - val_acc: 0.4525
Epoch 126/1000
100800/100800 [==============================] - 1s - loss: 0.6410 - acc: 0.7749 - val_loss: 3.0218 - val_acc: 0.4507
Epoch 127/1000
100800/100800 [==============================] - 1s - loss: 0.6479 - acc: 0.7727 - val_loss: 3.0155 - val_acc: 0.4515
Epoch 128/1000
100800/100800 [==============================] - 1s - loss: 0.6446 - acc: 0.7719 - val_loss: 2.9998 - val_acc: 0.4517
Epoch 129/1000
100800/100800 [==============================] - 1s - loss: 0.6455 - acc: 0.7729 - val_loss: 2.9871 - val_acc: 0.4531
Epoch 130/1000
100800/100800 [==============================] - 1s - loss: 0.6429 - 

100800/100800 [==============================] - 1s - loss: 0.6440 - acc: 0.7731 - val_loss: 2.9750 - val_acc: 0.4513
Epoch 246/1000
100800/100800 [==============================] - 1s - loss: 0.6427 - acc: 0.7729 - val_loss: 2.9638 - val_acc: 0.4537
Epoch 247/1000
100800/100800 [==============================] - 1s - loss: 0.6420 - acc: 0.7719 - val_loss: 2.9504 - val_acc: 0.4569
Epoch 248/1000
100800/100800 [==============================] - 1s - loss: 0.6388 - acc: 0.7740 - val_loss: 2.9508 - val_acc: 0.4572
Epoch 249/1000
100800/100800 [==============================] - 1s - loss: 0.6395 - acc: 0.7753 - val_loss: 2.9664 - val_acc: 0.4557
Epoch 250/1000
100800/100800 [==============================] - 1s - loss: 0.6433 - acc: 0.7732 - val_loss: 2.9755 - val_acc: 0.4556
Epoch 251/1000
100800/100800 [==============================] - 1s - loss: 0.6326 - acc: 0.7774 - val_loss: 2.9772 - val_acc: 0.4547
Epoch 252/1000
100800/100800 [==============================] - 1s - loss: 0.6414 - 

100800/100800 [==============================] - 1s - loss: 0.6312 - acc: 0.7783 - val_loss: 2.9839 - val_acc: 0.4558
Epoch 368/1000
100800/100800 [==============================] - 1s - loss: 0.6354 - acc: 0.7763 - val_loss: 2.9842 - val_acc: 0.4549
Epoch 369/1000
100800/100800 [==============================] - 1s - loss: 0.6367 - acc: 0.7761 - val_loss: 2.9906 - val_acc: 0.4546
Epoch 370/1000
100800/100800 [==============================] - 1s - loss: 0.6399 - acc: 0.7739 - val_loss: 2.9897 - val_acc: 0.4561
Epoch 371/1000
100800/100800 [==============================] - 1s - loss: 0.6329 - acc: 0.7765 - val_loss: 2.9691 - val_acc: 0.4577
Epoch 372/1000
100800/100800 [==============================] - 1s - loss: 0.6403 - acc: 0.7743 - val_loss: 2.9572 - val_acc: 0.4576
Epoch 373/1000
100800/100800 [==============================] - 1s - loss: 0.6413 - acc: 0.7752 - val_loss: 2.9718 - val_acc: 0.4554
Epoch 374/1000
100800/100800 [==============================] - 1s - loss: 0.6427 - 

100800/100800 [==============================] - 1s - loss: 0.6462 - acc: 0.7737 - val_loss: 2.9840 - val_acc: 0.4558
Epoch 490/1000
100800/100800 [==============================] - 1s - loss: 0.6393 - acc: 0.7742 - val_loss: 2.9874 - val_acc: 0.4543
Epoch 491/1000
100800/100800 [==============================] - 1s - loss: 0.6523 - acc: 0.7705 - val_loss: 2.9734 - val_acc: 0.4565
Epoch 492/1000
100800/100800 [==============================] - 1s - loss: 0.6435 - acc: 0.7724 - val_loss: 2.9771 - val_acc: 0.4575
Epoch 493/1000
100800/100800 [==============================] - 1s - loss: 0.6698 - acc: 0.7655 - val_loss: 2.9869 - val_acc: 0.4556
Epoch 494/1000
100800/100800 [==============================] - 1s - loss: 0.6456 - acc: 0.7720 - val_loss: 3.0075 - val_acc: 0.4511
Epoch 495/1000
100800/100800 [==============================] - 1s - loss: 0.6395 - acc: 0.7756 - val_loss: 3.0197 - val_acc: 0.4488
Epoch 496/1000
100800/100800 [==============================] - 1s - loss: 0.6407 - 

...

100800/100800 [==============================] - 1s - loss: 0.6349 - acc: 0.7761 - val_loss: 2.9271 - val_acc: 0.4554
Epoch 390/1000
100800/100800 [==============================] - 1s - loss: 0.6272 - acc: 0.7780 - val_loss: 2.9292 - val_acc: 0.4560
Epoch 391/1000
100800/100800 [==============================] - 1s - loss: 0.6339 - acc: 0.7765 - val_loss: 2.9170 - val_acc: 0.4570
Epoch 392/1000
100800/100800 [==============================] - 1s - loss: 0.6396 - acc: 0.7737 - val_loss: 2.9202 - val_acc: 0.4579
Epoch 393/1000
100800/100800 [==============================] - 1s - loss: 0.6403 - acc: 0.7743 - val_loss: 2.9291 - val_acc: 0.4576
Epoch 394/1000
100800/100800 [==============================] - 1s - loss: 0.6298 - acc: 0.7792 - val_loss: 2.9301 - val_acc: 0.4579
Epoch 395/1000
100800/100800 [==============================] - 1s - loss: 0.6383 - acc: 0.7761 - val_loss: 2.9185 - val_acc: 0.4584
Epoch 396/1000
100800/100800 [==============================] - 1s - loss: 0.6311 - 

100800/100800 [==============================] - 1s - loss: 0.6458 - acc: 0.7723 - val_loss: 2.9488 - val_acc: 0.4579
Epoch 512/1000
100800/100800 [==============================] - 1s - loss: 0.6381 - acc: 0.7746 - val_loss: 2.9547 - val_acc: 0.4548
Epoch 513/1000
100800/100800 [==============================] - 1s - loss: 0.6480 - acc: 0.7731 - val_loss: 2.9557 - val_acc: 0.4529
Epoch 514/1000
100800/100800 [==============================] - 1s - loss: 0.6353 - acc: 0.7770 - val_loss: 2.9428 - val_acc: 0.4530
Epoch 515/1000
100800/100800 [==============================] - 1s - loss: 0.6375 - acc: 0.7765 - val_loss: 2.9476 - val_acc: 0.4523
Epoch 516/1000
100800/100800 [==============================] - 1s - loss: 0.6318 - acc: 0.7789 - val_loss: 2.9445 - val_acc: 0.4525
Epoch 517/1000
100800/100800 [==============================] - 1s - loss: 0.6440 - acc: 0.7739 - val_loss: 2.9356 - val_acc: 0.4553
Epoch 518/1000
100800/100800 [==============================] - 1s - loss: 0.6343 - 

100800/100800 [==============================] - 1s - loss: 0.6437 - acc: 0.7742 - val_loss: 2.9638 - val_acc: 0.4565
Epoch 634/1000
100800/100800 [==============================] - 1s - loss: 0.6392 - acc: 0.7738 - val_loss: 2.9922 - val_acc: 0.4544
Epoch 635/1000
100800/100800 [==============================] - 1s - loss: 0.6409 - acc: 0.7742 - val_loss: 2.9968 - val_acc: 0.4511
Epoch 636/1000
100800/100800 [==============================] - 1s - loss: 0.6406 - acc: 0.7756 - val_loss: 2.9790 - val_acc: 0.4504
Epoch 637/1000
100800/100800 [==============================] - 1s - loss: 0.6329 - acc: 0.7759 - val_loss: 2.9575 - val_acc: 0.4514
Epoch 638/1000
100800/100800 [==============================] - 1s - loss: 0.6385 - acc: 0.7760 - val_loss: 2.9493 - val_acc: 0.4542
Epoch 639/1000
100800/100800 [==============================] - 1s - loss: 0.6417 - acc: 0.7720 - val_loss: 2.9445 - val_acc: 0.4561
Epoch 640/1000
100800/100800 [==============================] - 1s - loss: 0.6348 - 

100800/100800 [==============================] - 1s - loss: 0.6390 - acc: 0.7761 - val_loss: 2.9325 - val_acc: 0.4568
Epoch 756/1000
100800/100800 [==============================] - 1s - loss: 0.6342 - acc: 0.7762 - val_loss: 2.9167 - val_acc: 0.4583
Epoch 757/1000
100800/100800 [==============================] - 1s - loss: 0.6434 - acc: 0.7730 - val_loss: 2.9246 - val_acc: 0.4565
Epoch 758/1000
100800/100800 [==============================] - 1s - loss: 0.6337 - acc: 0.7755 - val_loss: 2.9448 - val_acc: 0.4563
Epoch 759/1000
100800/100800 [==============================] - 1s - loss: 0.6397 - acc: 0.7751 - val_loss: 2.9721 - val_acc: 0.4546
Epoch 760/1000
100800/100800 [==============================] - 1s - loss: 0.6313 - acc: 0.7758 - val_loss: 2.9671 - val_acc: 0.4555
Epoch 761/1000
100800/100800 [==============================] - 1s - loss: 0.6327 - acc: 0.7767 - val_loss: 2.9561 - val_acc: 0.4548
Epoch 762/1000
100800/100800 [==============================] - 1s - loss: 0.6327 - 

100800/100800 [==============================] - 1s - loss: 0.6341 - acc: 0.7761 - val_loss: 2.9707 - val_acc: 0.4561
Epoch 878/1000
100800/100800 [==============================] - 1s - loss: 0.6422 - acc: 0.7746 - val_loss: 2.9687 - val_acc: 0.4560
Epoch 879/1000
100800/100800 [==============================] - 1s - loss: 0.6402 - acc: 0.7733 - val_loss: 2.9562 - val_acc: 0.4556
Epoch 880/1000
100800/100800 [==============================] - 1s - loss: 0.6377 - acc: 0.7764 - val_loss: 2.9410 - val_acc: 0.4554
Epoch 881/1000
100800/100800 [==============================] - 1s - loss: 0.6349 - acc: 0.7748 - val_loss: 2.9329 - val_acc: 0.4563
Epoch 882/1000
100800/100800 [==============================] - 1s - loss: 0.6309 - acc: 0.7776 - val_loss: 2.9521 - val_acc: 0.4555
Epoch 883/1000
100800/100800 [==============================] - 1s - loss: 0.6365 - acc: 0.7749 - val_loss: 2.9687 - val_acc: 0.4540
Epoch 884/1000
100800/100800 [==============================] - 1s - loss: 0.6433 - 

In [96]:
model.save('./full_test/network_11_2000ep_59_42')

In [125]:
model = keras.models.load_model('./full_test/network_11_99000ep_59_4')

In [126]:
model.evaluate(samples_scaled,labels, batch_size=60000)

120000/126000 [===========================>..] - ETA: 0s

[0.75230106569471811, 0.85073090167272658]

In [104]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_156 (Dense)            (None, 2560)              97280     
_________________________________________________________________
dropout_65 (Dropout)         (None, 2560)              0         
_________________________________________________________________
batch_normalization_133 (Bat (None, 2560)              10240     
_________________________________________________________________
dense_157 (Dense)            (None, 1280)              3278080   
_________________________________________________________________
dropout_66 (Dropout)         (None, 1280)              0         
_________________________________________________________________
batch_normalization_134 (Bat (None, 1280)              5120      
_________________________________________________________________
dense_158 (Dense)            (None, 1280)              1639680   
__________

In [109]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[-6].output])
layer_output = get_3rd_layer_output([samples_scaled])[0]

InvalidArgumentError: You must feed a value for placeholder tensor 'dropout_1/keras_learning_phase' with dtype bool
	 [[Node: dropout_1/keras_learning_phase = Placeholder[dtype=DT_BOOL, shape=<unknown>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: dense_161_1/Relu/_8815 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_540_dense_161_1/Relu", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dropout_1/keras_learning_phase', defined at:
  File "/home/josh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/josh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/josh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/josh/a

In [110]:
def get_ith_layer_output(model, X, i, mode='test'):
    ''' see https://keras.io/getting-started/faq/#keras-faq-frequently-asked-keras-questions'''
    get_ith_layer = keras.backend.function(
        [model.layers[0].input, keras.backend.learning_phase()], [model.layers[i].output])
    layer_output = get_ith_layer([X, 0 if mode=='test' else 1])[0]
    return layer_output

In [112]:
layer = get_ith_layer_output(model,samples_scaled, -6)


In [117]:
np.save('layer_Results', layer)

In [128]:
np.save('test_set',test_ind)

In [116]:
%%time

from sklearn.manifold import TSNE

TSNE_model = TSNE(n_components=2, random_state=0, perplexity=40, n_iter=250)

TSNE_Results = TSNE_model.fit_transform(layer)

np.save('TSNE_Results', TSNE_Results)

CPU times: user 29min 4s, sys: 5min 48s, total: 34min 52s
Wall time: 34min 27s


In [ ]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[-2].output])
layer_output = get_3rd_layer_output([samples_scaled])[0]

In [ ]:
layer_output.shape

In [ ]:
from sklearn.manifold import TSNE

TSNE_model = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=1000)

%time
TSNE_Results = TSNE_model.fit_transform(layer_output)

np.save('TSNE_Results', TSNE_Results)

In [ ]:
np.save('TSNE_Results', TSNE_Results)

In [ ]:
tsne_df = pd.DataFrame(TSNE_Results, columns=['Component 1', 'Component 2'], index=df1.index)
data_all = pd.concat([df1, tsne_df], axis=1)

In [ ]:
category = 'System'

category_items = data_all[category].unique()

palette = []
for i in range(len(category_items)):
    palette = np.append(palette,"#%06x" % np.random.randint(0, 0xFFFFFF))
#palette = brewer['Set3'][len(category_items) + 1]
colormap = dict(zip(category_items, palette))
data_all['color'] = data_all[category].map(colormap)

In [ ]:
source = ColumnDataSource(data_all.iloc[test_ind])

hover = HoverTool(tooltips=[(column, '@' + column) for column in ['lattice_a', 'lattice_b', 'lattice_c', 'alfa', 'beta', 'gama','Energy']])

tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), TapTool(), SaveTool()]

p = figure(
    tools=tools,
    #title=title,
    plot_width=800,
    plot_height=800,
    toolbar_location='below',
    toolbar_sticky=False, )

p.circle(
    x='Component 1',
    y='Component 2',
    source=source,
    size=10,
    line_color='#333333',
    line_width=0.5,
    fill_alpha=0.8,
    color='color',
    legend=category)

show(p)

In [ ]:
np.save('test_ind',test_ind)

In [ ]:
data_all.to_pickle('TSNE_DataFrame')

In [ ]:
# Input - type the file name
file = 'Data_V2.csv'

# loads the data
df1 = pd.read_csv(file)

# Prints the sheet names
#print(x1.sheet_names)

# Selects the sheet store
#df1 = x1.parse('Total Data Set')

# Splits the arrays based on the atoms
elements = df1['System'].str.findall('[A-Z][^A-Z]*')
elements = pd.DataFrame(elements.values.tolist(), columns = ['A_cation','B_cation','Oxygen'])

In [ ]:
# Converts the text labels into examples
encoder = LabelEncoder()
encoder.fit(elements['A_cation'])
a_cation_one_hot = encoder.transform(elements['A_cation'])

# convert integers to dummy variables (i.e. one hot encoded)
a_labels = np_utils.to_categorical(a_cation_one_hot)

encoder = LabelEncoder()
encoder.fit(elements['B_cation'])
B_cation_one_hot = encoder.transform(elements['B_cation'])

# convert integers to dummy variables (i.e. one hot encoded)
b_labels = np_utils.to_categorical(B_cation_one_hot)

# Collects the data for the samples
samples = df1.as_matrix(columns=df1.columns[1:])

# Standard scaler to the raw data
samples_scaled = StandardScaler().fit_transform(samples)

Input_data = np.column_stack((samples_scaled[:,0:21], a_labels, b_labels))
Output_data = samples_scaled[:,22]

train, test = train_test_split(np.linspace(
                0, Input_data.shape[0] - 1, Input_data.shape[0] - 1).astype(int), test_size=.2)

In [ ]:
df1.columns.values

In [ ]:
# First small model

# establishes the model
model = Sequential()


# Dense layer size 16
model.add(Dense(256, input_dim=42, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense layer size 16
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 16
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

# Dense layer size 8
model.add(Dense(2, activation='relu'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())
#
# Dense softmax layer for classification
model.add(Dense(1, activation='linear'))

#opt = Adam()

# Compiles the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

#model.optimizer.lr.assign(0.0001)
hist = model.fit(Input_data[train], Output_data[train], epochs=5000, 
             batch_size=53091, validation_data = (Input_data[test],Output_data[test]))#,callbacks=[tbCallBack])

In [ ]:
model.save('./just_energy_5000e_val_009')


In [ ]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[-1].output])
layer_output = get_3rd_layer_output([Input_data])[0]

In [ ]:
def get_ith_layer_output(model, X, i, mode='test'):
    ''' see https://keras.io/getting-started/faq/#keras-faq-frequently-asked-keras-questions'''
    get_ith_layer = keras.backend.function(
        [model.layers[0].input, keras.backend.learning_phase()], [model.layers[i].output])
    layer_output = get_ith_layer([X, 0 if mode=='test' else 1])[0]
    return layer_output

In [ ]:
layer_output = get_ith_layer_output(model, Input_data, -2)

In [ ]:
np.save('layer_output', layer_output)

In [ ]:
plt.scatter(layer_output[:,1],layer_output[:,0])

In [ ]:
model.predict(Input_data[test])

In [ ]:
# Collects the data for the samples
samples = df1.as_matrix(columns=df1.columns[1:38])
df1['Energy']

In [ ]:
test

In [ ]:
# Standard scaler to the raw data
energy = StandardScaler()

stan = energy.fit_transform(samples[:,21].reshape(-1,1))

In [ ]:
energy.inverse_transform(Output_data[test].reshape(-1,1))

In [ ]:
# Converts the text labels into examples
encoder = LabelEncoder()
encoder.fit(df1['System'])
encoded_labels = encoder.transform(df1['System'])

# convert integers to dummy variables (i.e. one hot encoded)
labels = np_utils.to_categorical(encoded_labels)

# Collects the data for the samples
samples = df1.as_matrix(columns=df1.columns[1:38])

# Standard scaler to the raw data
samples_scaled=StandardScaler().fit_transform(samples)

In [ ]:
# define 10-fold cross validation test harness
kfold = KFold(n_splits=, shuffle=True, random_state=10)
cvscores = []
for train, test in kfold.split(samples_scaled, labels):
    
    # establishes the model
    model = Sequential()
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model.add(Dense(64, input_dim=37))
    
    # Dense layer size 32
    model.add(Dense(32, activation='relu'))
    
    # Dense layer size 16
    model.add(Dense(16, activation='relu'))
    
    # Dense layer size 8
    model.add(Dense(8, activation='relu'))
    
    # Dense softmax layer for classification
    model.add(Dense(54, activation='softmax'))
    
    # Compiles the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fits the model using the training set
    model.fit(samples_scaled[train], labels[train], epochs=10000, batch_size=1000, verbose=0)
    
    # evaluates the model
    scores = model.evaluate(samples_scaled[test], labels[test], verbose=0)
    
    # prints the scores
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
model_big = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model_big.add(Dense(512, input_dim=37))

# Dense layer size 64, # input dimensions based on the size of the input data
model_big.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Dense layer size 64, # input dimensions based on the size of the input data
model_big.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Dense layer size 32
model_big.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Dense layer size 16
model_big.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Dense layer size 8
model_big.add(Dense(16, activation='relu'))

# Dense softmax layer for classification
model_big.add(Dense(54, activation='softmax'))

# Compiles the model
model_big.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# establishes the model
model = Sequential()

# Dense layer size 64, # input dimensions based on the size of the input data
model.add(Dense(128, input_dim=37))
model.add(Dropout(0.5))

# Dense layer size 32
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Dense layer size 16
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Dense layer size 8
model.add(Dense(8, activation='relu'))

# Dense softmax layer for classification
model.add(Dense(54, activation='softmax'))

# Compiles the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_big.optimizer.lr=1e-2
model_big.fit(samples_scaled, labels, epochs=5000, 
             batch_size=53091, validation_split=.2, shuffle=True)

In [ ]:
samples_scaled.shape

In [ ]:
# define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=10)
cvscores = []
for train, test in kfold.split(samples_scaled, labels):
    
    # establishes the model
    model = Sequential()
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model.add(Dense(128, input_dim=37))
    
    # Dense layer size 32
    model.add(Dense(64, activation='relu'))
    
    # Dense layer size 16
    model.add(Dense(32, activation='relu'))
    
    # Dense layer size 8
    model.add(Dense(16, activation='relu'))
    
    # Dense softmax layer for classification
    model.add(Dense(4, activation='softmax'))
    
    # Compiles the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fits the model using the training set
    model.fit(samples_scaled[train], labels[train], epochs=1000, batch_size=1000, verbose=0)
    
    # evaluates the model
    scores = model.evaluate(samples_scaled[test], labels[test], verbose=0)
    
    # prints the scores
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
model.summary()

In [ ]:
scores

In [ ]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[3].output])
layer_output = get_3rd_layer_output([samples_scaled])[0]

In [ ]:
layer_output

In [ ]:
from sklearn.manifold import TSNE

TSNE_model = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=1000)

%time
TSNE_Results = TSNE_model.fit_transform(layer_output)

In [ ]:
tsne_df = pd.DataFrame(TSNE_Results, columns=['Component 1', 'Component 2'], index=df1.index)

In [ ]:
data_all = pd.concat([df1, tsne_df], axis=1)

In [ ]:
category = 'System'

category_items = data_all[category].unique()
palette = brewer['Set3'][len(category_items) + 1]
colormap = dict(zip(category_items, palette))
data_all['color'] = data_all[category].map(colormap)

In [ ]:
data_all.head

In [ ]:
source = ColumnDataSource(data_all[:])

hover = HoverTool(tooltips=[(column, '@' + column) for column in ['lattice_a', 'lattice_b', 'lattice_c', 'alfa', 'beta', 'gama','Energy']])

tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), TapTool(), SaveTool()]

p = figure(
    tools=tools,
    #title=title,
    plot_width=800,
    plot_height=800,
    toolbar_location='below',
    toolbar_sticky=False, )

p.circle(
    x='Component 1',
    y='Component 2',
    source=source,
    size=10,
    line_color='#333333',
    line_width=0.5,
    fill_alpha=0.8,
    color='color',
    legend=category)

show(p)

In [ ]:
Systems = ['PbTiO3','PbZrO3','KNbO3','NaVO3']
Pand_Name = ['Index','PbTiO3','PbZrO3','KNbO3','NaVO3']


data_sort = np.zeros((4,1000))
for i, name in enumerate(Systems):
    data_sort[i] = data_all['Energy'][data_all['System']==name][0:1000].values

In [ ]:
sorted_df = pd.DataFrame(data_sort[:,data_sorted[0]].T,columns=Pand_Name[1:5])
sorted_df = pd.merge(sorted_df, data_all[0:1000], right_index=True, left_index=True)
index_df = pd.DataFrame({'Index':np.linspace(0,999,1000)})
sorted_df.sort_values('PbTiO3')
sorted_df = sorted_df.join(index_df)

In [ ]:
source = ColumnDataSource(sorted_df)

hover = HoverTool(tooltips=[(column, '@' + column) for column in ['lattice_a', 'lattice_b', 'lattice_c', 'alfa', 'beta', 'gama','Energy']])

tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), TapTool(), SaveTool()]

p = figure(
    tools=tools,
    #title=title,
    plot_width=800,
    plot_height=800,
    toolbar_location='below',
    toolbar_sticky=False, )
for i, name in enumerate(Systems):
    p.circle(
        x='Index',
        y=name,
        source=source,
        size=8,
        line_color='#333333',
        line_width=0.5,
        fill_alpha=0.8,
        color=palette[i],
        legend=name+' ')

#p.circle(
#    x='Index',
#    y='PbZrO3',
#    source=source,
#    size=3,
#    line_color='#333333',
#    line_width=0.5,
#    fill_alpha=0.8,
#    color='color',
#    legend='PbZrO3 ')

show(p)

In [ ]:
sorted_df.head()

In [ ]:
data_all[1:1001].values[data_sorted]

In [ ]:
df5 = pd.DataFrame([1, 2, 3, 4, 5], index=[100, 29, 234, 1, 150], columns=['A'])
df5.sort_index(inplace=True)
print(df5.to_string())

In [ ]:
# define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=10)
cvscores = []
for train, test in kfold.split(samples_scaled, labels):
    
    # establishes the model
    model_big = Sequential()
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model_big.add(Dense(128, input_dim=37))
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model_big.add(Dense(128, activation='relu'))
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model_big.add(Dense(64, activation='relu'))
    
    # Dense layer size 32
    model_big.add(Dense(32, activation='relu'))
    
    # Dense layer size 16
    model_big.add(Dense(16, activation='relu'))
    
    # Dense layer size 8
    model_big.add(Dense(8, activation='relu'))
    
    # Dense softmax layer for classification
    model_big.add(Dense(4, activation='softmax'))
    
    # Compiles the model
    model_big.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fits the model using the training set
    model_big.fit(samples_scaled[train], labels[train], epochs=10000, batch_size=1000, verbose=0)
    
    # evaluates the model
    scores = model_big.evaluate(samples_scaled[test], labels[test], verbose=0)
    
    # prints the scores
    print("%s: %.2f%%" % (model_big.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
cvscores

In [ ]:
model_big.summary()

In [ ]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model_big.layers[0].input],
                                  [model_big.layers[5].output])
layer_output = get_3rd_layer_output([samples_scaled])[0]

In [ ]:
from sklearn.manifold import TSNE

TSNE_model_2 = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=2000)

%time
TSNE_Results_2 = TSNE_model.fit_transform(layer_output)

In [ ]:
tsne_df = pd.DataFrame(TSNE_Results_2, columns=['Component 1', 'Component 2'], index=df1.index)
data_all = pd.concat([df1, tsne_df], axis=1)
category = 'System'

category_items = data_all[category].unique()
palette = brewer['Set3'][len(category_items) + 1]
colormap = dict(zip(category_items, palette))
data_all['color'] = data_all[category].map(colormap)

In [ ]:
source = ColumnDataSource(data_all[:])

hover = HoverTool(tooltips=[(column, '@' + column) for column in ['lattice_a', 'lattice_b', 'lattice_c', 'alfa', 'beta', 'gama','Energy']])

tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), TapTool(), SaveTool()]

p = figure(
    tools=tools,
    #title=title,
    plot_width=800,
    plot_height=800,
    toolbar_location='below',
    toolbar_sticky=False, )

p.circle(
    x='Component 1',
    y='Component 2',
    source=source,
    size=10,
    line_color='#333333',
    line_width=0.5,
    fill_alpha=0.8,
    color='color',
    legend=category)

show(p)

In [ ]:
data_all.head()

In [ ]:
# define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=10)
cvscores = []
for train, test in kfold.split(samples_scaled, labels):
    
    # establishes the model
    model_2 = Sequential()
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model_2.add(Dense(128, input_dim=37))
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model_2.add(Dense(128, activation='relu'))
    
    # Dense layer size 64, # input dimensions based on the size of the input data
    model_2.add(Dense(64, activation='relu'))
    
    # Dense layer size 32
    model_2.add(Dense(32, activation='relu'))
    
    # Dense layer size 16
    model_2.add(Dense(16, activation='relu'))
    
    # Dense layer size 8
    model_2.add(Dense(2, activation='linear'))
    
    # Dense softmax layer for classification
    model_2.add(Dense(4, activation='softmax'))
    
    # Compiles the model
    model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fits the model using the training set
    model_2.fit(samples_scaled[train], labels[train], epochs=10000, batch_size=1000, verbose=0)
    
    # evaluates the model
    scores = model_2.evaluate(samples_scaled[test], labels[test], verbose=0)
    
    # prints the scores
    print("%s: %.2f%%" % (model_2.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
cvscores

In [ ]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model_2.layers[0].input],
                                  [model_2.layers[5].output])
layer_output = get_3rd_layer_output([samples_scaled])[0]

In [ ]:
tsne_df = pd.DataFrame(layer_output, columns=['Component 1', 'Component 2'], index=df1.index)
data_all = pd.concat([df1, tsne_df], axis=1)
category = 'System'

category_items = data_all[category].unique()
palette = brewer['Set3'][len(category_items) + 1]
colormap = dict(zip(category_items, palette))
data_all['color'] = data_all[category].map(colormap)

In [ ]:
source = ColumnDataSource(data_all[:])

hover = HoverTool(tooltips=[(column, '@' + column) for column in ['lattice_a', 'lattice_b', 'lattice_c', 'alfa', 'beta', 'gama','Energy']])

tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), TapTool(), SaveTool()]

p = figure(
    tools=tools,
    #title=title,
    plot_width=800,
    plot_height=800,
    toolbar_location='below',
    toolbar_sticky=False, )

p.circle(
    x='Component 1',
    y='Component 2',
    source=source,
    size=10,
    line_color='#333333',
    line_width=0.5,
    fill_alpha=0.8,
    color='color',
    legend=category)

show(p)

In [ ]:
model_2.save('model_2d_linear')

In [ ]:
model.save('model_1')

In [ ]:
model_big.save('model_big')

In [ ]:
np.savez_compressed('all_data')